[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/comp-neural-circuits/plasticity-workshop/blob/dev/rate_based.ipynb)

# Rate-based Plasticity Rules

## Hebbian Plasticity

**Goals**
+ Covariance-based learning rule is equivalent to detecting the first principal component of the activity


### Initialization

In [ ]:
!pip install numpy scipy matplotlib ipywidgets scikit-learn --quiet
import numpy as np
import scipy.linalg as lin
from numpy.random import default_rng
rng = default_rng()
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#plt.style.use("https://github.com/comp-neural-circuits/plasticity-workshop/raw/dev/plots_style.txt")
plt.style.use("plots_style.txt")

### Utility Functions

In [ ]:
def ornstein_uhlenbeck(mean,cov,dt,Ttot,dts=1E-2):
  """
  Generates a multi-dimensional Ornstein-Uhlenbeck process.

  Parameters :
  mean (numpy vector) : desired mean
  cov  (matrix)   : covariance matrix (symmetric, positive definite)
  dt   (real)     : timestep output
  Tot  (real)     : total time
  dts = 1E-3 (real) : simulation timestep

  Returns :
  times (numpy vector)
  rates (numpy matrix)  :  rates[i,j] is the rate of unit i at time times[j]
  """
  times = np.linspace(0.0,Ttot-dt,num=int(Ttot/dt))
  n = len(mean)
  nTs = int(Ttot/dts)
  rates_all = np.empty((n,nTs))
  rates_all[:,0] = mean
  L = lin.cholesky(cov)
  nskip = int(dt/dts)
  assert round(dts*nskip,5) == dt , "dt must be multiple of  " + str(dts)
  for t in range(1,nTs):
    dr = dts*(mean-rates_all[:,t-1])
    dpsi = np.sqrt(2*dts)*(L.T @ rng.standard_normal(n))
    rates_all[:,t] = rates_all[:,t-1] + dr + dpsi
  # subsample 
  rates = rates_all[:,::nskip]
  return times,rates
  
def twodimensional_UL(mean1,var1,mean2,var2,corr,dt,Ttot,dts=1E-2):
  """
  Generates samples from a 2D Ornstein-Uhlenbeck process.

  Parameters :
  mean1 (real) : mean on first dimension
  var1  (real) : variance on first dimension (at dt=1. intervals)
  mean2 (real) : - 
  var2  (real) : - 
  corr  (real) : correlation coefficient 
  dt   (real)     : timestep output
  Tot  (real)     : total time
  dts = 1E-3 (real) : simulation timestep

  Returns :
  times  (numpy vector)
  rates1 (numpy vector)
  rates2 (numpy vector)
  """
  assert -1<=corr<=1, "correlation must be in (-1,1) interval"
  var12 = corr*np.sqrt(var1*var2)
  (times, rates) = ornstein_uhlenbeck(
      np.array([mean1,mean2]),
      np.array([[var1,var12],[var12,var2]]),
      dt,Ttot,dts)
  return times, rates[0,:],rates[1,:]


def plot_r1_and_r2(correlation=0.0,mean_r1=0.0,mean_r2=0.0,var_r1=1.0,var_r2=1.0):
    times,rates1,rates2 = twodimensional_UL(mean_r1,var_r1,mean_r2,var_r2,correlation,0.1,60.0)
    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,10)) #gridspec_kw={'height_ratios': [3, 1]})
    ax1.plot(times,rates1)
    ax1.plot(times,rates2)
    ax1.set_xlabel("time (s)")
    ax1.set_ylabel("rate (Hz)")
    ax1.set_title("time traces")
    ax2.scatter(rates1,rates2,color="black")
    ax2.set_title("samples r1 Vs r2")
    ax2.set_xlabel("rate 1 (Hz)")
    ax2.set_ylabel("rate 2 (Hz)")
    ax2.axis("equal")
    return 

### Visualize noisy rate inputs

In this tutorial we generate a noisy rate trace from $N$ neurons. $r_j(t)$ indicates the rate of neuron $j=1,2,\ldots\,N$ at time $t$. Neurons are, in general, correlated with each other.

In the figure below, you can see the traces of 2 neurons, simulated for 60 seconds. You don't need to read or understand this code. Try to modify some of the parameters to understand their behavior.

In [ ]:
interact(plot_r1_and_r2,mean_r1=(0.0,5.0,0.1),mean_r2=(0.0,5.0,0.1),
        var_r1=(0.01,2.0,0.01), var_r2=(0.01,2.0,0.01));


### Compute response of output neuron based on input activity and weights  (exercise 1 ?)

$$
r_\text{out}(t) = \sum_{j=1}^N w_j r_j(t)
$$

In [ ]:
def rate_response(r_input,weights):
    """
    Computes the response of a neuron that receives a series of inputs over time.  
    
    Parameters :
    r_input (matrix) :  r_input[i,t] is the rate of input neuron i at timestep t
    weights (vector) :  weights[i] is the synaptic strenght between neuron i and the output neuron
    
    Returns :
    r_output (vector) : r_output[t] is the rate of the output neuron at timestep t
    """
    
    # I think this can be done with np.dot , but I don't like to see it applied to matrices
    # so I propose a more canonical broadcasting
    
    r_input_weighted = r_input * weights[:,np.newaxis] # multiply columnwise
    r_output = r_input_weighted.sum(axis=0) # and sum columnwise
    return r_output

### Compute weight update for correlation based and covariance based rule

Now that you have the reponse of the output neuron  $r_\text{out}(t)$ , you can calculate the update in synaptic weights due to rate-based plasticiy.


For the correlation-based rule, we have :
$$ 
\Delta w_j = \gamma \; \left<  r_\text{out}(t) \; r_j(t)  \right>_t
$$


For the covariance based rule, we are using a covariance, instead :
$$ 
\Delta w_j = \gamma \; \left<  \left(r_\text{out}(t) - \bar{r}_\text{out}\right)
\; \left(r_j(t) - \bar{r}_j \right)  \right>_t \quad \text{with} \quad 
\bar{r}_\text{out} = \left< r_\text{out}(t) \right>_t \quad \text{and} \quad
\bar{r}_j = \left< r_j(t) \right>_t 
$$




In [ ]:
def weight_updates_correlation(r_input,weights_current,gamma):
    """
    Computes the weight updates according to the correlation rule
    
    Parameters :
    r_input (matrix) :  r_input[i,t] is the rate of input neuron i at timestep t
    weights_current (vector) :  weights[i] is the synaptic strenght between neuron i and the output neuron
    
    Returns :
    weight_updates (vector) : the update on each weight after this training interval
    """

    # TIPS : 
    # use the rate_response function that you defined before !
   
    r_output = rate_response(r_input,weights_current)
    r_product  = r_output * r_input  # broadcast by row
    weight_updates = gamma * r_product.mean(axis=1) # mean over time dimension
    return weights_update

In [ ]:
def weight_updates_covariance(r_input,weights_current,gamma):
    """
    Computes the weight updates according to the covariance rule
    
    Parameters :
    r_input (matrix) :  r_input[i,t] is the rate of input neuron i at timestep t
    weights_current (vector) :  weights[i] is the synaptic strenght between neuron i and the output neuron
    
    Returns :
    weight_updates (vector) : the update on each weight after this training interval
    """
    
    # TIPS : 
    # it is very similar to the correlation rule, except you need to subtract the mean rates!
    # r_input_means = r_input.mean(axis=1)  # (mean over time axis)
    r_output_mean = r_output.mean() # mean of a vector -> scalar value
    r_output = rate_response(r_input,weights_current)
    r_output_meanzero = r_output - r_output_mean
    r_input_means = r_input.mean(axis=1) # mean over time axis
    r_input_meanzero = r_input - r_input_means[:,np.newaxis] # broadcast on columns
    
    # now same as before
    r_product = r_output_meanzero * r_input_meanzero
    weight_updates = gamma * r_product.mean(axis=1) # mean over time dimension
    return weights_update

